In [1]:
import os
def find_repo_root(startpath):
    current_path = os.path.abspath(startpath) # Path started on
    while True:
        if os.path.isdir(os.path.join(current_path, '.git')) or os.path.isfile(os.path.join(current_path, 'README.md')): # If on git path, return it
            return current_path
        
        parent_path = os.path.dirname(current_path)

        if parent_path == current_path: # If current path is parent path, stop 
            break
        current_path = parent_path # Set current path to parent path, to check if git path again
    return None # If an issue arises 
root = find_repo_root(os.getcwd())
        

In [2]:
import pandas as pd
df = pd.read_csv(f"{root}/Data/liteweight/2020_genre_counts_by_trope.csv", index_col=0) # Make dataframe out of genre-trope csv, without unnamed column, 
df = df[["Trope", "Associated_Movies", "Number_movies"]] # Drop all columns except for the specified ones
total_movies = sum(df["Number_movies"]) 
df["Probability"] = df["Number_movies"] / total_movies # Create new column with probability of trope being in a given movie
df.head()


,Trope,Associated_Movies,Number_movies,Probability
0,ABNegative,"['Andhadhun', 'Blood Work', 'Bruce Almighty', ...",10,0.000019
1,ABirthdayNotABreak,"['Batman Begins', 'Beast', 'Big Game', 'Creatu...",13,0.000025
2,ABloodyMess,"['Carry on Behind', 'Cloud Atlas', 'Cornered',...",12,0.000023
3,ABoyAGirlAndABabyFamily,"['Honey, I Blew Up the Kid', 'The Addams Famil...",3,0.000006
4,ABoyAndHisX,"['Air Bud', 'Alpha', 'Bumblebee', 'Fido', 'Fly...",13,0.000025


In [5]:
# This is code for Alien specifically, should be adapted for any tropeogram
movie_tropes = pd.read_csv(f"{root}/Code/Alien Stuff/Alien Tropes - CodeTest.tsv", delimiter='\t')
movie_tropes["Trope"] = movie_tropes["Trope"].apply(lambda x: x.replace(" ", "")) # Remove spaces from tropes
movie_tropes = df.where(df["Trope"].isin(movie_tropes["Trope"])) # Filter out anything not in movie
movie_tropes.dropna(inplace = True) # Remove everything filtered out
movie_tropes.drop("Associated_Movies", axis = 1, inplace = True) # Drop extra column for readability
movie_tropes.head()


,Trope,Number_movies,Probability
62,AIIsACrapshoot,86.0,0.000165
316,ActionSurvivor,166.0,0.000318
414,AdmiringTheAbomination,30.0,0.000058
560,AlienBlood,63.0,0.000121
571,AlienSky,37.0,0.000071


In [3]:
import numpy as np 
def H(list_of_probs): # Shannon entropy, sum over each trope in the movie. Other definitions of entropy pending
    entropy = 0
    for prob in list_of_probs:
        if prob == 0:
            pass
        else:
            entropy -= prob * np.log10(prob)
    return entropy

In [27]:
import csv
def find_tropes(movie):
    with open(f"{root}/Data/movies-vs-tropes.csv") as file: # Pick out a movie and returns a lists of tropes
        reader = csv.reader(file)
        for line in reader:
            line = list(line)
            if line[0].split(";")[0] == movie: # Deal with formatting, search lines of the csv until the movie is found
                film = line[0].split(";")[0]
                trope_1 = line[0].split(";")[1]
                line.pop(0)
                line.append(trope_1)
                return line
            else: pass
    return None # Error if no movie found  
movie = "TheDictator" # Find tropes associated with a given movie 
tropes = find_tropes(movie)
movie_tropes = df.where(df["Trope"].isin(tropes)) # Filter out df
movie_tropes.dropna(inplace = True) # Remove everything filtered out
movie_tropes.drop("Associated_Movies", axis = 1, inplace = True) # Drop extra column for readability
movie_tropes.sort_values(by='Number_movies', ascending=False, inplace=True)
movie_tropes.sort_values(by='Number_movies', ascending=False, inplace=True)

movie_tropes.head(15)

cutoff_movie = movie_tropes[movie_tropes['Number_movies'] < 1000]
cutoff_movie.head(10)

,Trope,Number_movies,Probability
12145,KarmaHoudini,961.0,0.001844
22231,TheCameo,937.0,0.001798
1432,AssholeVictim,889.0,0.001705
2897,BrickJoke,835.0,0.001602
11944,JerkWithAHeartOfGold,754.0,0.001446
6846,EvenEvilHasStandards,636.0,0.001220
22970,TheReasonYouSuckSpeech,629.0,0.001207
3106,ButtMonkey,627.0,0.001203
2494,BlatantLies,602.0,0.001155
15543,NotSoDifferent,587.0,0.001126


In [28]:
entropy = H(cutoff_movie["Probability"]) # This is for baseline entropy, NOT genre dependent
print(entropy)
# Sort of a measure of how unexpected a movie is, higher value for lesser used tropes, test more popular films 

0.1613820290943652


In [9]:
# To do - Identify genre at each time stamp and use it to find entropy values at each time stamp
# Ex. Entropy for 'Chestburster' is likely much higher if it's within a Romance movie
# Make probability a function of genre, genre is a function of time, so entropy is a function of time
trope_genredf = pd.read_csv(f"{root}/Data/liteweight/2020_genre_counts_by_trope.csv")
trope_genredf.head()
genre = "Action" # This will wind up being replaced by the genre at a given timestamp
# Need to access total number of movies for each genre 

In [ ]:
# Find total number of movies for each genre
# Use other models of entropy?
def find_number_of_genre(genre): # Finds number of movies associated with each genre
    movies = [] 
    with open(f"{root}/Data/movies-vs-tropes.csv") as file:
        reader = csv.reader(file)
        reader.__next__() # Skip first line
        for line in reader:
            movies.append(line[0])
    movies = [movie.split(";")[0].lower() for movie in movies] # Get all movies used in entropy calculation
    imdb_df = pd.read_csv(f'{root}/Data/IMBD_Dataframe.csv') # Match with IMDB data
    imdb_df['name'] = imdb_df['name'].apply(lambda x: str(x)) # Turn movie column into strings with no spaces
    imdb_df['name'] = imdb_df['name'].str.replace(" ", "")
    imdb_df['name'] = imdb_df['name'].apply(lambda x:x.lower())
    imdb_df = imdb_df[imdb_df['name'].isin(movies)] # So far this is a script to cut out all movies NOT included in the tropes data
    # Past this point is finding the genre
    imdb_df['genre'] = imdb_df['genre'].apply(lambda x:x.replace("[", '').replace(']', '')) # String formatting, SHOULD BE REDONE WITH string.translate
    imdb_df['genre'] = imdb_df['genre'].apply(lambda x:x.replace("'", ""))
    imdb_df['genre'] = imdb_df['genre'].apply(lambda x:list(x.split(","))) # Turn genre section into list
    def contains_genre(list_):
        if genre in list_:
            return 1
        else:
            return 0
    imdb_df['contained?'] = imdb_df['genre'].apply(lambda x:contains_genre(x))
    return sum(imdb_df['contained?'])


161


In [ ]:
# Instead of using total movies, use the number of movies tagged with specific genre, then the probability becomes the associated movies / number found from function
# This should take less than a minute to run
def genrentropy(movie, genre):
    genre_dependednt_df = pd.read_csv(f"{root}/Data/liteweight/2020_genre_counts_by_trope.csv", index_col=0) # This was opened earlier but for a different purpose
    genre_dependednt_df['Probability'] = genre_dependednt_df[genre] / find_number_of_genre(genre) # Add probability   
    genre_dependednt_df = genre_dependednt_df.where(genre_dependednt_df['Probability'] != 0).dropna() # Remove all zeros so that entropy is well-defined
    tropes = find_tropes(movie)
    genre_dependednt_df = genre_dependednt_df.where(genre_dependednt_df["Trope"].isin(tropes)) # Keep in only the tropes from movie
    genre_dependednt_df.dropna(inplace = True) # Remove everything filtered out
    genre_dependednt_df = genre_dependednt_df[['Trope', genre, 'Probability']]
    #genre_dependednt_df.head()
    genre_entropy = H(genre_dependednt_df['Probability'])
    return genre_entropy
genrentropy_ = genrentropy('Jumanji', 'Action')

7.797482239838537


In [151]:
genrestring = 'Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,Game-Show,History,Horror,Music,Musical,Mystery,News,Romance,Sci-Fi,Short,Sport,Thriller,War,Western'
genre_list = genrestring.split(',') # Including this because I didn't feel like opening another df globally just for column names
genrentropy_values =[]
for genre in genre_list:
    genrentropy_values.append(genrentropy('GoodWillHunting', genre))
# Takes a few minutes, if error in movie title, should be recognized within 10s

In [153]:
import plotly.express as px
plot_df = pd.DataFrame({'Genre':genre_list, 'Entropy':genrentropy_values})
fig = px.bar(plot_df, x = 'Genre', y = 'Entropy', title = 'Genre-based entropy for Good Will Hunting')
fig.show()


In [150]:
up = find_tropes('TheEmojiMovie')
print(up)

None
